In [2]:
#!pip install yfinance

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
import os
import yfinance as yf

In [7]:
def margin_req(strike,price,premium,n_options = 100, type = "put"):
    if type == "put":
        # let's find the maximum between 3 choices:
        max_choice = np.max([(0.3 * price - np.max([price - strike,0])) + premium,
               (0.1 * strike) + premium,
               2.50 + premium])
        return max_choice  * n_options * 100
    elif type == "call":
         # let's find the maximum between 3 choices:
        max_choice = np.max([(0.3 * price - np.max([-(price - strike),0])) + premium,
               (0.1 * strike) + premium,
               2.50 + premium])
        return max_choice * n_options * 100


In [5]:
margin_req(90,100,1,n_options=1,type="put")

1100.0

In [8]:
# checking the margin requirement for a put option that evolves with the price of the underlying
price_evol = [100,95,90,85,40,50]
premium_evol = [1,2,4,7,50.5,40]
for i in range(0,len(price_evol)):
    print(margin_req(90,price_evol[i],premium_evol[i],n_options=1,type="put"))

2100.0
2550.0
3100.0
3250.0
6250.0
5500.0


In [9]:
# checking the margin requirement for a call option that evolves with the price of the underlying
price_evol = [40,45,50,55,90,100]
premium_evol = [1,2,4,7,40.5,50]
for i in range(0,len(price_evol)):
    print(margin_req(50,price_evol[i],premium_evol[i],n_options=1,type="call"))

600.0
1050.0
1900.0
2350.0
6750.0
8000.0


In [10]:
pwd

'c:\\Users\\HP\\source\\repos\\optionstrade'

In [11]:
position_jan_11_25 = pd.read_csv("data/positions_11_01_2025.csv", engine='python', encoding = "ISO-8859-1")

In [12]:
position_jan_11_25.head()

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75
1,SPDR S&P 500 ETF TRUST,NaN,USD,USD,USD,USD,USD,NaN
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73
3,ISHARES BITCOIN TRUST ETF,NaN,USD,USD,USD,USD,USD,NaN
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25


In [13]:
position_jan_11_25["Instrument\xa0"]

0            SPY Jan17'25 585 Put 
1           SPDR S&P 500 ETF TRUST
2            IBIT Mar21'25 51 Put 
3        ISHARES BITCOIN TRUST ETF
4            SPY Mar21'25 545 Put 
5           SPDR S&P 500 ETF TRUST
6          CAR.UN Feb21'25 48 Put 
7     CAN APARTMENT PROP REAL ESTA
8            IBIT Jan17'25 55 Put 
9        ISHARES BITCOIN TRUST ETF
10           IWM Jan17'25 219 Put 
11        ISHARES RUSSELL 2000 ETF
12          MSFT Jan17'25 420 Put 
13                  MICROSOFT CORP
14          NVDA Jan17'25 136 Put 
15                     NVIDIA CORP
16          RCAT Apr17'25 15 Call 
17            RED CAT HOLDINGS INC
18           CRNT Mar21'25 6 Call 
19            CERAGON NETWORKS LTD
20          IONQ Apr17'25 55 Call 
21                        IONQ INC
22         NVDA Apr17'25 150 Call 
23                     NVIDIA CORP
24         ANET Mar21'25 120 Call 
25             ARISTA NETWORKS INC
26                           IBIT 
27       ISHARES BITCOIN TRUST ETF
28         RDDT Mar2

In [14]:
position_jan_11_25["type"] = position_jan_11_25["Instrument\xa0"].str.split().str[-1]

In [16]:
position_jan_11_25.head()

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put
1,SPDR S&P 500 ETF TRUST,NaN,USD,USD,USD,USD,USD,NaN,TRUST
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put
3,ISHARES BITCOIN TRUST ETF,NaN,USD,USD,USD,USD,USD,NaN,ETF
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put


In [18]:
position_jan_11_25["type0"] = ""
for y in range(0,len(position_jan_11_25["type"])):
    #for i in position_jan_11_25["type"].unique():
    if position_jan_11_25["type"][y] == "Put":
        position_jan_11_25["type0"][y] = "option" 
    elif position_jan_11_25["type"][y] == "Call":
        position_jan_11_25["type0"][y] = "option"
    else: 
        position_jan_11_25["type0"][y] = "Not option"

In [19]:
position_jan_11_25

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option
1,SPDR S&P 500 ETF TRUST,NaN,USD,USD,USD,USD,USD,NaN,TRUST,Not option
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option
3,ISHARES BITCOIN TRUST ETF,NaN,USD,USD,USD,USD,USD,NaN,ETF,Not option
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option
5,SPDR S&P 500 ETF TRUST,NaN,USD,USD,USD,USD,USD,NaN,TRUST,Not option
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",C5.55,Put,option
7,CAN APARTMENT PROP REAL ESTA,NaN,CAD,CAD,CAD,CAD,CAD,NaN,ESTA,Not option
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option
9,ISHARES BITCOIN TRUST ETF,NaN,USD,USD,USD,USD,USD,NaN,ETF,Not option


In [20]:
option_position_jan_11_25 = position_jan_11_25[position_jan_11_25["type0"] == "option"]

In [21]:
option_position_jan_11_25.head(10)

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",C5.55,Put,option
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option
10,IWM Jan17'25 219 Put,-3,-643.45,-611,"-1,357.62",2.14,-651,4.14,Put,option
12,MSFT Jan17'25 420 Put,-2,-659.54,-420,"-1,094.96",3.3,-414,5.45,Put,option
14,NVDA Jan17'25 136 Put,-3,-608.62,-343,"-1,010.76",2.03,-367,3.2,Put,option
16,RCAT Apr17'25 15 Call,20,"8,995.57","-5,380.00","3,617.67",4.5,"-1,590.00",1.8,Call,option
18,CRNT Mar21'25 6 Call,200,"12,157","-6,480.00","5,706.23",0.61,-822,0.3,Call,option


In [22]:
option_position_jan_11_25["Instrument\xa0"].apply(lambda st: st[st.find("25")+2:-1].split()[0])

0     585
2      51
4     545
6      48
8      55
10    219
12    420
14    136
16     15
18      6
20     55
22    150
24    120
28    180
Name: Instrument , dtype: object

In [23]:
option_position_jan_11_25["Strike"] = option_position_jan_11_25["Instrument\xa0"].apply(lambda st: st[st.find("25")+2:-1].split()[0])

C:\Users\HP\AppData\Local\Temp\ipykernel_7216\2233995446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_position_jan_11_25["Strike"] = option_position_jan_11_25["Instrument\xa0"].apply(lambda st: st[st.find("25")+2:-1].split()[0])


In [24]:
option_position_jan_11_25.head()

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option,585
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option,51
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option,545
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",C5.55,Put,option,48
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option,55


In [25]:
option_position_jan_11_25["underlying"] = option_position_jan_11_25["Instrument\xa0"].apply(lambda st: st.split()[0])

C:\Users\HP\AppData\Local\Temp\ipykernel_7216\247712363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_position_jan_11_25["underlying"] = option_position_jan_11_25["Instrument\xa0"].apply(lambda st: st.split()[0])


In [26]:
option_position_jan_11_25.head()

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike,underlying
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option,585,SPY
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option,51,IBIT
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option,545,SPY
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",C5.55,Put,option,48,CAR.UN
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option,55,IBIT


In [27]:
# Define the ticker list
tickers_list = option_position_jan_11_25["underlying"].unique()
# Create placeholder for data
options_underlying_jan_11_25 = pd.DataFrame(columns=tickers_list)
# Fetch the data
for ticker in tickers_list:
    options_underlying_jan_11_25[ticker] = yf.download(ticker,
                               start="2025-01-11",
                               end="2025-01-11")['Close']
# Print first 5 rows of the data
options_underlying_jan_11_25.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CAR.UN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,SPY,IBIT,CAR.UN,IWM,MSFT,NVDA,RCAT,CRNT,IONQ,ANET,RDDT
Date,,,,,,,,,,,
2025-01-10,580.48999,53.84,NaN,216.830002,418.950012,135.910004,10.03,4.81,32.330002,114.339996,167.080002


In [28]:
options_underlying_jan_11_25 = options_underlying_jan_11_25.transpose()

In [29]:
options_underlying_jan_11_25.head()

Date,2025-01-10
SPY,580.489990
IBIT,53.840000
CAR.UN,NaN
IWM,216.830002
MSFT,418.950012


In [30]:
options_underlying_jan_11_25.columns = ["underlying_price"]

In [31]:
options_underlying_jan_11_25

,underlying_price
SPY,580.489990
IBIT,53.840000
CAR.UN,NaN
IWM,216.830002
MSFT,418.950012
NVDA,135.910004
RCAT,10.030000
CRNT,4.810000
IONQ,32.330002
ANET,114.339996


In [32]:
options_underlying_jan_11_25.reset_index(drop=False, inplace=True)

In [33]:
options_underlying_jan_11_25

,index,underlying_price
0,SPY,580.489990
1,IBIT,53.840000
2,CAR.UN,NaN
3,IWM,216.830002
4,MSFT,418.950012
5,NVDA,135.910004
6,RCAT,10.030000
7,CRNT,4.810000
8,IONQ,32.330002
9,ANET,114.339996


In [34]:
options_underlying_jan_11_25.rename(columns={"index":"underlying"}, inplace=True)

In [35]:
options_underlying_jan_11_25

,underlying,underlying_price
0,SPY,580.489990
1,IBIT,53.840000
2,CAR.UN,NaN
3,IWM,216.830002
4,MSFT,418.950012
5,NVDA,135.910004
6,RCAT,10.030000
7,CRNT,4.810000
8,IONQ,32.330002
9,ANET,114.339996


In [36]:
options_underlying_jan_11_25.loc[pd.isna(options_underlying_jan_11_25.underlying_price),"underlying_price"] = 41.71

In [37]:
options_underlying_jan_11_25

,underlying,underlying_price
0,SPY,580.489990
1,IBIT,53.840000
2,CAR.UN,41.710000
3,IWM,216.830002
4,MSFT,418.950012
5,NVDA,135.910004
6,RCAT,10.030000
7,CRNT,4.810000
8,IONQ,32.330002
9,ANET,114.339996


In [38]:
option_position_jan_11_25

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike,underlying
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option,585,SPY
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option,51,IBIT
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option,545,SPY
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",C5.55,Put,option,48,CAR.UN
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option,55,IBIT
10,IWM Jan17'25 219 Put,-3,-643.45,-611,"-1,357.62",2.14,-651,4.14,Put,option,219,IWM
12,MSFT Jan17'25 420 Put,-2,-659.54,-420,"-1,094.96",3.3,-414,5.45,Put,option,420,MSFT
14,NVDA Jan17'25 136 Put,-3,-608.62,-343,"-1,010.76",2.03,-367,3.2,Put,option,136,NVDA
16,RCAT Apr17'25 15 Call,20,"8,995.57","-5,380.00","3,617.67",4.5,"-1,590.00",1.8,Call,option,15,RCAT
18,CRNT Mar21'25 6 Call,200,"12,157","-6,480.00","5,706.23",0.61,-822,0.3,Call,option,6,CRNT


In [39]:
option_position_jan_11_25["\xa0Position"] = pd.to_numeric(option_position_jan_11_25["\xa0Position"], errors='coerce')
short_option_position_jan_11_25 = option_position_jan_11_25[option_position_jan_11_25["\xa0Position"] < 0]

C:\Users\HP\AppData\Local\Temp\ipykernel_7216\935677338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_position_jan_11_25["\xa0Position"] = pd.to_numeric(option_position_jan_11_25["\xa0Position"], errors='coerce')


In [40]:
short_option_position_jan_11_25["\xa0Last"] = short_option_position_jan_11_25["\xa0Last"].astype('str').str.extractall('([0-9]*+[./]*[0-9]*)').unstack().fillna('').sum(axis=1)
short_option_position_jan_11_25

C:\Users\HP\AppData\Local\Temp\ipykernel_7216\843502164.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_option_position_jan_11_25["\xa0Last"] = short_option_position_jan_11_25["\xa0Last"].astype('str').str.extractall('([0-9]*+[./]*[0-9]*)').unstack().fillna('').sum(axis=1)


,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike,underlying
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option,585,SPY
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option,51,IBIT
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option,545,SPY
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",5.55,Put,option,48,CAR.UN
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option,55,IBIT
10,IWM Jan17'25 219 Put,-3,-643.45,-611,"-1,357.62",2.14,-651,4.14,Put,option,219,IWM
12,MSFT Jan17'25 420 Put,-2,-659.54,-420,"-1,094.96",3.3,-414,5.45,Put,option,420,MSFT
14,NVDA Jan17'25 136 Put,-3,-608.62,-343,"-1,010.76",2.03,-367,3.20,Put,option,136,NVDA


In [41]:
short_option_position_jan_11_25.columns

Index(['Instrument ', ' Position', ' Cost Basis', ' Unrealized P&L',
       ' Market Value', ' Avg Price', ' Daily P&L', ' Last', 'type', 'type0',
       'Strike', 'underlying'],
      dtype='object')

In [42]:
short_option_position_jan_11_25[['\xa0Cost Basis', '\xa0Unrealized P&L', '\xa0Avg Price', '\xa0Daily P&L', 'Strike','\xa0Market Value']] = short_option_position_jan_11_25[['\xa0Cost Basis', '\xa0Unrealized P&L', '\xa0Avg Price', '\xa0Daily P&L', 'Strike','\xa0Market Value']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',',''), errors='coerce'))

C:\Users\HP\AppData\Local\Temp\ipykernel_7216\927802322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_option_position_jan_11_25[['\xa0Cost Basis', '\xa0Unrealized P&L', '\xa0Avg Price', '\xa0Daily P&L', 'Strike','\xa0Market Value']] = short_option_position_jan_11_25[['\xa0Cost Basis', '\xa0Unrealized P&L', '\xa0Avg Price', '\xa0Daily P&L', 'Strike','\xa0Market Value']].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',',''), errors='coerce'))


In [43]:
# Attach underlying price to the short option position
short_option_position_jan_11_25 = pd.merge(short_option_position_jan_11_25, options_underlying_jan_11_25, on="underlying")
short_option_position_jan_11_25

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike,underlying,underlying_price
0,SPY Jan17'25 585 Put,-27,-8513.10,-12030.0,-21455.60,3.15,-11150.0,7.75,Put,option,585,SPY,580.489990
1,SPY Mar21'25 545 Put,-20,-8565.64,-3970.0,-12691.09,4.28,-3560.0,6.25,Put,option,545,SPY,580.489990
2,IBIT Mar21'25 51 Put,-50,-17722.00,-931.0,-18866.18,3.54,1850.0,3.73,Put,option,51,IBIT,53.840000
3,IBIT Jan17'25 55 Put,-60,-9170.94,-3190.0,-12360.81,1.53,3130.0,2.01,Put,option,55,IBIT,53.840000
4,CAR.UN Feb21'25 48 Put,-20,-5375.00,-7260.0,-12636.27,2.69,-1540.0,5.55,Put,option,48,CAR.UN,41.710000
5,IWM Jan17'25 219 Put,-3,-643.45,-611.0,-1357.62,2.14,-651.0,4.14,Put,option,219,IWM,216.830002
6,MSFT Jan17'25 420 Put,-2,-659.54,-420.0,-1094.96,3.30,-414.0,5.45,Put,option,420,MSFT,418.950012
7,NVDA Jan17'25 136 Put,-3,-608.62,-343.0,-1010.76,2.03,-367.0,3.20,Put,option,136,NVDA,135.910004


In [44]:
short_option_position_jan_11_25["\xa0Last"].dtype

dtype('float64')

In [45]:
short_option_position_jan_11_25["margin_requirement"] = short_option_position_jan_11_25.apply(lambda x: margin_req(x["Strike"],x["underlying_price"],x["\xa0Last"],n_options=abs(x["\xa0Position"]),type=x["type"].lower()), axis=1)
short_option_position_jan_11_25

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike,underlying,underlying_price,margin_requirement
0,SPY Jan17'25 585 Put,-27,-8513.10,-12030.0,-21455.60,3.15,-11150.0,7.75,Put,option,585,SPY,580.489990,491121.892090
1,SPY Mar21'25 545 Put,-20,-8565.64,-3970.0,-12691.09,4.28,-3560.0,6.25,Put,option,545,SPY,580.489990,289814.013672
2,IBIT Mar21'25 51 Put,-50,-17722.00,-931.0,-18866.18,3.54,1850.0,3.73,Put,option,51,IBIT,53.840000,85209.999466
3,IBIT Jan17'25 55 Put,-60,-9170.94,-3190.0,-12360.81,1.53,3130.0,2.01,Put,option,55,IBIT,53.840000,108972.000275
4,CAR.UN Feb21'25 48 Put,-20,-5375.00,-7260.0,-12636.27,2.69,-1540.0,5.55,Put,option,48,CAR.UN,41.710000,36126.000000
5,IWM Jan17'25 219 Put,-3,-643.45,-611.0,-1357.62,2.14,-651.0,4.14,Put,option,219,IWM,216.830002,20756.700165
6,MSFT Jan17'25 420 Put,-2,-659.54,-420.0,-1094.96,3.30,-414.0,5.45,Put,option,420,MSFT,418.950012,26227.000732
7,NVDA Jan17'25 136 Put,-3,-608.62,-343.0,-1010.76,2.03,-367.0,3.20,Put,option,136,NVDA,135.910004,13191.900330


In [46]:
short_option_position_jan_11_25["portfolio_margin_requirement"] = short_option_position_jan_11_25["margin_requirement"].sum()
short_option_position_jan_11_25

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0,Strike,underlying,underlying_price,margin_requirement,portfolio_margin_requirement
0,SPY Jan17'25 585 Put,-27,-8513.10,-12030.0,-21455.60,3.15,-11150.0,7.75,Put,option,585,SPY,580.489990,491121.892090,1.071420e+06
1,SPY Mar21'25 545 Put,-20,-8565.64,-3970.0,-12691.09,4.28,-3560.0,6.25,Put,option,545,SPY,580.489990,289814.013672,1.071420e+06
2,IBIT Mar21'25 51 Put,-50,-17722.00,-931.0,-18866.18,3.54,1850.0,3.73,Put,option,51,IBIT,53.840000,85209.999466,1.071420e+06
3,IBIT Jan17'25 55 Put,-60,-9170.94,-3190.0,-12360.81,1.53,3130.0,2.01,Put,option,55,IBIT,53.840000,108972.000275,1.071420e+06
4,CAR.UN Feb21'25 48 Put,-20,-5375.00,-7260.0,-12636.27,2.69,-1540.0,5.55,Put,option,48,CAR.UN,41.710000,36126.000000,1.071420e+06
5,IWM Jan17'25 219 Put,-3,-643.45,-611.0,-1357.62,2.14,-651.0,4.14,Put,option,219,IWM,216.830002,20756.700165,1.071420e+06
6,MSFT Jan17'25 420 Put,-2,-659.54,-420.0,-1094.96,3.30,-414.0,5.45,Put,option,420,MSFT,418.950012,26227.000732,1.071420e+06
7,NVDA Jan17'25 136 Put,-3,-608.62,-343.0,-1010.76,2.03,-367.0,3.20,Put,option,136,NVDA,135.910004,13191.900330,1.071420e+06
